In [1]:
import os
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project")
print(os.getcwd())

/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project


In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as ps
import glob
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [3]:
states = pd.read_csv("data/States.csv")

In [4]:
NumCensusAreas = pd.read_csv("StatesNumCensusAreas.csv", header=0)
NumCensusAreas.head()

,State,NumCensusAreas
0,alabama,67
1,alaska,29
2,arizona,15
3,arkansas,75
4,california,58


In [5]:
def cap(string):
    if isinstance(string, str):
        return string.upper().replace(" COUNTY", "").capitalize().strip()
    else:
        return ""

In [6]:
#--------------------READING AND PRE-PROCESSING POWER PLANT OUTPUT DATA-------------------
#-----------------------------------------------------------------------------------------

def get_plant():
    files = glob.glob("data/*2_3_4_5*")

    for i in range(len(files)):
        print(files[i])
        if i == 0:
            data2 = get_pplant(files[i])
        else:
            upd = get_pplant(files[i])
            data2 = pd.concat([data2, upd])
    
    return data2


def get_pplant(filename):
    xls = pd.ExcelFile(filename)
    data2 = xls.parse("Page 1 Generation and Fuel Data", skiprows=5, index_col=None, na_values=[np.nan])

    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    
    #Initializing with January
    data2_t = data2.iloc[:, [0, 96, 6, 55, 79]]
    data2_t.columns = ["Plant ID", "Year", "Plant State", "Tot_Consumed_MMBtu", "Net_Gen_MWh"]
    data2_t["Month"] = 1
    
    #Transposing against month
    for i in range(1, 12):
        #print(i)
        data_m = data2.iloc[:, [0, 96, 6, 55 + i, 79 + i]]
        data_m.columns = ["Plant ID", "Year", "Plant State", "Tot_Consumed_MMBtu", "Net_Gen_MWh"]
        data_m["Month"] = months[i]
        data2_t = pd.concat([data2_t, data_m])
    
    return data2_t

#Getting plant data
data2 = get_plant()

data/EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_12_2016_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_12_2017_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_12_2015_Final_Revision.xlsx
data/EIA923_Schedules_2_3_4_5_M_08_2018_25OCT2018.xlsx


In [7]:
#Obtaining plant location data for operating plants
xls = pd.ExcelFile("data/august_generator2018.xlsx")
plant_loc = xls.parse("Operating", skiprows=1, index_col=None, na_values=[np.nan])
q1 = """SELECT DISTINCT `Plant ID`, `Technology`, `Energy Source Code`, County FROM plant_loc"""
plant_loc = ps.sqldf(query=q1)

#Obtaining full state name from state abbreviation
states_p = states.copy()
states_p.columns = ["State", "Plant State"]
states_p["State"] = states_p["State"].apply(str.capitalize)
data2 = pd.merge(data2, states_p, how="left", on=["Plant State"])

In [8]:
#Obtaining plant location data for retired plants
xls2 = pd.ExcelFile("data/august_generator2018.xlsx")
plant_loc2 = xls2.parse("Retired", skiprows=1, index_col=None, na_values=[np.nan])
q2 = """SELECT DISTINCT `Plant ID`, `Technology`, `Energy Source Code`, County FROM plant_loc2"""
plant_loc2 = ps.sqldf(query=q2)
plant_loc2.head()

,Plant ID,Technology,Energy Source Code,County
0,58422.0,Other Natural Gas,NG,Los Angeles
1,55554.0,Landfill Gas,LFG,Galveston
2,57786.0,Landfill Gas,LFG,Frederick
3,995.0,Conventional Steam Coal,BIT,Porter
4,1002.0,Petroleum Liquids,DFO,Fayette


In [9]:
#Merging in plant location from State
data3 = pd.merge(data2, plant_loc, how="left", on=["Plant ID"])
data3 = pd.merge(data3, plant_loc2, how="left", on=["Plant ID"])

data3['County'] = data3['County_x'].where(data3['County_x'].notnull(), data3['County_y'])
data3['Technology'] = data3['Technology_x'].where(data3['Technology_x'].notnull(), data3['Technology_y'])
data3['Energy Source Code'] = data3['Energy Source Code_x'].where(data3['Energy Source Code_x'].notnull(), data3['Energy Source Code_y'])
data3.drop(['County_x','County_y','Energy Source Code_x', 'Energy Source Code_y','Technology_x', 'Technology_y'], axis=1, inplace=True)
display(data3.head())

,Plant ID,Year,Plant State,Tot_Consumed_MMBtu,Net_Gen_MWh,Month,State,County,Technology,Energy Source Code
0,2,2013,AL,227955,23892.1,1,Alabama,Tuscaloosa,Conventional Hydroelectric,WAT
1,3,2013,AL,11278,192108,1,Alabama,Mobile,Natural Gas Steam Turbine,NG
2,3,2013,AL,11278,192108,1,Alabama,Mobile,Conventional Steam Coal,BIT
3,3,2013,AL,11278,192108,1,Alabama,Mobile,Natural Gas Fired Combined Cycle,NG
4,3,2013,AL,3832458,358682,1,Alabama,Mobile,Natural Gas Steam Turbine,NG


In [10]:
#Removing trailing blanks from county and state variables and make lowercase
data3["County"] = data3["County"].apply(cap).str.lower()
data3["State"] = data3["State"].apply(cap).str.lower()

#Fix "sout" by replacing with "south dakota" (consistent with Plant State)
data3['State'] = data3['State'].replace('sout', 'south dakota')

#Replace "washington dc" with "district of columbia" to be consistent with population data
data3['State'] = data3['State'].replace('washington dc', 'district of columbia')

#Replace . with NaNs
data3['Tot_Consumed_MMBtu'] = data3['Tot_Consumed_MMBtu'].replace('.', np.nan)
data3['Net_Gen_MWh'] = data3['Net_Gen_MWh'].replace('.', np.nan)
data3['County'] = data3['County'].replace('', np.nan)

#Add a category for State-Fuel Level Increment
data3["State-Fuel Level Increment"] = 1 * (data3['Plant ID'] == 99999)
display(data3.tail())

,Plant ID,Year,Plant State,Tot_Consumed_MMBtu,Net_Gen_MWh,Month,State,County,Technology,Energy Source Code,State-Fuel Level Increment
1245355,99999,2018,WI,NaN,NaN,12,wisconsin,NaN,NaN,NaN,1
1245356,99999,2018,WI,NaN,NaN,12,wisconsin,NaN,NaN,NaN,1
1245357,99999,2018,WV,NaN,NaN,12,west virginia,NaN,NaN,NaN,1
1245358,99999,2018,WY,NaN,NaN,12,wyoming,NaN,NaN,NaN,1
1245359,99999,2018,WY,NaN,NaN,12,wyoming,NaN,NaN,NaN,1


In [11]:
data3_missing = data3[data3['County'].isnull()]
data3_missing.head()

statefuel = data3_missing[data3_missing['State-Fuel Level Increment'] == 1]

print('# missing counties =', len(data3_missing))
print('# missing counties, state-fuel level increment =', len(statefuel))
print('delta missing =', len(data3_missing) - len(statefuel))
print('% missing =', round((len(data3_missing) - len(statefuel))*100/len(data3),2),'%')
print(len(data3))

# missing counties = 36072
# missing counties, state-fuel level increment = 31512
delta missing = 4560
% missing = 0.37 %
1245360


In [12]:
powerplant = data3

In [24]:
query = '''SELECT * FROM powerplant WHERE Year = 2016'''
powerplant_2016 = ps.sqldf(query=query)
powerplant_2016.head()
print(len(powerplant_2016))

198732


In [14]:
#Checking Number of States
query = """SELECT DISTINCT State FROM powerplant ORDER BY State ASC"""
len(ps.sqldf(query=query))

51

In [15]:
#Checking number of key variable combinations
query = """SELECT COUNT(*) AS COUNT FROM (SELECT DISTINCT State, County, Year FROM powerplant)"""
ps.sqldf(query=query)

,COUNT
0,12656


In [16]:
#Checking which powerplants are missing data
query = """SELECT [Plant ID] FROM powerplant WHERE County IS NULL"""
ps.sqldf(query=query)

,Plant ID
0,3823
1,7939
2,8812
3,8812
4,8816
5,8816
6,8816
7,8823
8,8823
9,8824


In [17]:
#Checking Number of Counties for each State
query = """SELECT State, COUNT(DISTINCT County) as NumCensusAreas from powerplant GROUP BY State ORDER BY State ASC"""
check = ps.sqldf(query=query)

In [18]:
display(check[check['NumCensusAreas'] != NumCensusAreas['NumCensusAreas']].head())
check2 = check.copy()
check2['diff'] = check['NumCensusAreas'] - NumCensusAreas['NumCensusAreas']
display(check2[check2['diff'] > 0]) #many of these are retired so there are blanks

,State,NumCensusAreas
0,alabama,42
1,alaska,27
2,arizona,13
3,arkansas,36
4,california,55


,State,NumCensusAreas,diff


In [19]:
#Aggregating by county
query = """ SELECT Year, Month, State, County, SUM(Tot_Consumed_MMBtu) as pp_consumed_MMBtu, SUM(Net_Gen_MWh) as pp_net_gen_MWh
            FROM powerplant
            GROUP BY State, County, Year, Month 
            ORDER BY State ASC"""
powerplant_agg = ps.sqldf(query=query)

In [20]:
powerplant_agg[10000:10200]

,Year,Month,State,County,pp_consumed_MMBtu,pp_net_gen_MWh
10000,2014,5,california,inyo,1136298.0,119484.497
10001,2014,6,california,inyo,1107400.0,116445.879
10002,2014,7,california,inyo,1106846.0,116387.593
10003,2014,8,california,inyo,1094389.0,115077.512
10004,2014,9,california,inyo,1040381.0,109398.735
10005,2014,10,california,inyo,1038010.0,109149.364
10006,2014,11,california,inyo,1050841.0,110498.568
10007,2014,12,california,inyo,1091493.0,114773.171
10008,2015,1,california,inyo,1075220.0,115374.253
10009,2015,2,california,inyo,947060.0,101621.211


In [23]:
data3.to_csv("./cleaned_data/powerplant.csv", index=None, header=True)
data3_missing.to_csv("./cleaned_data/powerplant_missing-counties.csv", index=None, header=True)
powerplant_agg.to_csv("./cleaned_data/powerplant_agg.csv", index=None, header=True)
powerplant_2016.to_csv("./cleaned_data/powerplant_2016.csv", index=None, header=True)